## get_audio_features

Function takes song title, artist name, list of music features and returns a dataframe with audio features 

In [4]:
from bs4 import BeautifulSoup

import requests #allows to download the code of the webpage
import pandas as pd
import numpy as np

import sys
from Credentials import * 

import spotipy #spotify API 
import json
from spotipy.oauth2 import SpotifyClientCredentials

from  time import sleep


from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import pickle

import sys

import webbrowser
from IPython.display import Markdown, display

In [5]:
from Credentials import *



In [6]:
def clean_artist(artist_name):
    '''removes characters from artist name that are
    contained in break_list 
    i.e. 
    break_list = [ ' featuring ', ' x ', ' & ', 'with']'''
        
    break_list = [ ' featuring ', ' x ', ' & ', ' with']
    for break_point in break_list:
        artist_name = artist_name.replace(break_point,' ')
    
    return artist_name

In [7]:
def  get_audio_features(song_name, artist, music_features):
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID,
                                                           client_secret= Client_Secret))
    artist = clean_artist(artist)
    
    #form query for the song with the artist filer
    results = sp.search(q = ("artist:"+artist+' track:'+song_name))
    
    #taking item [0] as best match 
    #returns dict within a list; dictionary is one element of the list - hence [0] in the end
    features = (sp.audio_features(results["tracks"]["items"][0]["uri"]))[0]
    
    #only leaving the features we want - music_features
    unwanted =  set(features) - set(music_features)
    for unwanted_key in unwanted: 
        del features[unwanted_key]
    
    #to create df after - replacing values with lists
    for key, value in features.items():
        value_list = []
        value_list.append(value)
        features[key] = value_list
    
    feature_dataframe = pd.DataFrame(features)
    
    return feature_dataframe

In [8]:
song_name = 'rain on me'
artist = 'lady gaga'
music_features = ['danceability', 'energy', 'key',
  'loudness',
  'mode',
  'speechiness',
  'acousticness',
  'instrumentalness',
  'liveness',
  'valence',
  'tempo', 'id']

get_audio_features(song_name, artist, music_features)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.672,0.855,9,-3.764,1,0.0397,0.021,0,0.323,0.646,123.056,24ySl2hOPGCDcxBxFIqWBu


## PRODUCTION:

In [1]:
#pip install spotipy

## Loading credentials from another config file

In [44]:
import sys
#sys.path.insert(1, '/Credentials') #when not in the same folder as notebook
from Credentials import * #name of file 

## Starting with Spotify API

In [45]:
import spotipy #spotify API 
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [46]:
#Initialize SpotiPy with user credentias
# From: https://developer.spotify.com/documentation/general/guides/app-settings/ 
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID,
                                                           client_secret= Client_Secret))

# The "sp" variable has two useful funtions:
# The first usefull function is:
# .search(q='',limit=n)
# Where the "q" keyword is the query you want to perform on spotify: song_name, artist,...
# while The "limit" keyword will limit the number of returned results.
#
# The second usefull function is:
# .audio_features([URL|URI|ID])
# which returns some 'features of the song', that after cleanup, we can use in order to characterize a song.

In [53]:
song_name = 'rain on me'
artist = 'lady gaga'
query = song_name+' artist:'+artist
query

'rain on me artist:lady gaga'

In [50]:
results = sp.search(q='rain on me artist:lady gaga')
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=rain+on+me+artist%3Alady+gaga&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1HY2Jd0NmPuamShAr6KMms'},
       'href': 'https://api.spotify.com/v1/artists/1HY2Jd0NmPuamShAr6KMms',
       'id': '1HY2Jd0NmPuamShAr6KMms',
       'name': 'Lady Gaga',
       'type': 'artist',
       'uri': 'spotify:artist:1HY2Jd0NmPuamShAr6KMms'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR'},
       'href': 'https://api.spotify.com/v1/artists/66CXWjxzNUsdJxJ2JdwvnR',
       'id': '66CXWjxzNUsdJxJ2JdwvnR',
       'name': 'Ariana Grande',
       'type': 'artist',
       'uri': 'spotify:artist:66CXWjxzNUsdJxJ2JdwvnR'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',


In [54]:
results = sp.search(q=query)
#results
json_results = json.dumps(results, ensure_ascii=False)
json_results

'{"tracks": {"href": "https://api.spotify.com/v1/search?query=rain+on+me+artist%3Alady+gaga&type=track&offset=0&limit=10", "items": [{"album": {"album_type": "single", "artists": [{"external_urls": {"spotify": "https://open.spotify.com/artist/1HY2Jd0NmPuamShAr6KMms"}, "href": "https://api.spotify.com/v1/artists/1HY2Jd0NmPuamShAr6KMms", "id": "1HY2Jd0NmPuamShAr6KMms", "name": "Lady Gaga", "type": "artist", "uri": "spotify:artist:1HY2Jd0NmPuamShAr6KMms"}, {"external_urls": {"spotify": "https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR"}, "href": "https://api.spotify.com/v1/artists/66CXWjxzNUsdJxJ2JdwvnR", "id": "66CXWjxzNUsdJxJ2JdwvnR", "name": "Ariana Grande", "type": "artist", "uri": "spotify:artist:66CXWjxzNUsdJxJ2JdwvnR"}], "available_markets": ["AD", "AE", "AL", "AR", "AT", "AU", "BA", "BE", "BG", "BH", "BO", "BR", "BY", "CA", "CH", "CL", "CO", "CR", "CY", "CZ", "DE", "DK", "DO", "DZ", "EC", "EE", "EG", "ES", "FI", "FR", "GB", "GR", "GT", "HK", "HN", "HR", "HU", "ID", "IE", "IL

In [144]:
results["tracks"]["items"][0]["uri"]

'spotify:track:24ySl2hOPGCDcxBxFIqWBu'

In [145]:
features = sp.audio_features(results["tracks"]["items"][0]["uri"])
#returns dict within a list; dictionary is one element of the list

In [157]:
x = (sp.audio_features(results["tracks"]["items"][0]["uri"]))[0]

In [146]:
features = features[0]

In [147]:
features

{'danceability': 0.672,
 'energy': 0.855,
 'key': 9,
 'loudness': -3.764,
 'mode': 1,
 'speechiness': 0.0397,
 'acousticness': 0.021,
 'instrumentalness': 0,
 'liveness': 0.323,
 'valence': 0.646,
 'tempo': 123.056,
 'type': 'audio_features',
 'id': '24ySl2hOPGCDcxBxFIqWBu',
 'uri': 'spotify:track:24ySl2hOPGCDcxBxFIqWBu',
 'track_href': 'https://api.spotify.com/v1/tracks/24ySl2hOPGCDcxBxFIqWBu',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/24ySl2hOPGCDcxBxFIqWBu',
 'duration_ms': 182200,
 'time_signature': 4}

In [148]:
features.keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [149]:
music_features = ['danceability', 'energy', 'key',
  'loudness',
  'mode',
  'speechiness',
  'acousticness',
  'instrumentalness',
  'liveness',
  'valence',
  'tempo']
#the ones we are interested in 

In [150]:
#deleting unneeded 
unwanted =  set(features) - set(music_features)
for unwanted_key in unwanted: 
    del features[unwanted_key]

In [151]:
features
#that we can use to fit into the database 

{'danceability': 0.672,
 'energy': 0.855,
 'key': 9,
 'loudness': -3.764,
 'mode': 1,
 'speechiness': 0.0397,
 'acousticness': 0.021,
 'instrumentalness': 0,
 'liveness': 0.323,
 'valence': 0.646,
 'tempo': 123.056}

In [152]:
features['key']

9

In [153]:
f = features.copy()
f

{'danceability': 0.672,
 'energy': 0.855,
 'key': 9,
 'loudness': -3.764,
 'mode': 1,
 'speechiness': 0.0397,
 'acousticness': 0.021,
 'instrumentalness': 0,
 'liveness': 0.323,
 'valence': 0.646,
 'tempo': 123.056}

In [154]:
for key, value in f.items():
    x = []
    x.append(value)
    print(x)
    f[key] = x
    
f

[0.672]
[0.855]
[9]
[-3.764]
[1]
[0.0397]
[0.021]
[0]
[0.323]
[0.646]
[123.056]


{'danceability': [0.672],
 'energy': [0.855],
 'key': [9],
 'loudness': [-3.764],
 'mode': [1],
 'speechiness': [0.0397],
 'acousticness': [0.021],
 'instrumentalness': [0],
 'liveness': [0.323],
 'valence': [0.646],
 'tempo': [123.056]}

In [155]:
x = pd.DataFrame(f)

In [156]:
x

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.672,0.855,9,-3.764,1,0.0397,0.021,0,0.323,0.646,123.056
